# Optimization 2

Because one optimization notebook is never enough

In [1]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from matplotlib.animation import FuncAnimation

from IPython.core.display import Image, display
from mpl_toolkits.mplot3d import Axes3D

# Set plotting font sizes and properties
TINY_SIZE = 12
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 20
MARKER_SIZE = 6
LINE_SIZE = 4

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=TINY_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc("lines", markersize=MARKER_SIZE)  # marker size
plt.rc("lines", linewidth=LINE_SIZE)  # line width

mpl.rcParams["figure.dpi"] = 180 # sets the image quality

# Height and width per row and column of subplots
FIG_HEIGHT = 18
FIG_WIDTH = 16
fig_fcn = lambda kwargs: plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT), **kwargs)
color_list = sns.color_palette("Paired")

# Level Method

asdf

In [ ]:
# level method over the maxquad functions
class LevelMethod():
    
    def __init__(self, dim=10, k=5, C=1e-10, max_iter=1000):
        """
        :param: dim - int, size of matrices A_k 
        :param: k - int, number of matrices A_1,\dots, A_k
        :param: C - float, learning rate constant C: eta = C / sqrt{t}
        :param: max_iter - int, max number of iterations allowed
        """
        self.dim = int(dim)
        self.k = int(k)
        self.C = C
        self.max_iter = int(max_iter)
        
        # starting guess for optimum
        self.start = np.ones(dim)
        
        # initialize the set of matrices A_k
        self.A = {}
        for i in range(k):
            A_i = np.zeros(shape=(dim, dim))
            for m in range(dim):
                for n in range(i):
                    A_i[m, n] = np.exp((n + 1) / (m + 1)) * np.cos((m + 1) * (n + 1)) * np.sin((i + 1))
                    A_i[n, m] = A_i[m, n]
                A_i[m, m] = (i / 10) * np.abs(np.sin(i)) + np.sum(np.abs(A_i[m, :m])) + np.sum(np.abs(A_i[m, m+1:]))
            self.A[i] = A_i

        # initialize the vectors b_k
        self.b = {}
        for i in range(k):
            b_i = np.zeros(dim)
            for j in range(dim):
                b_i[j] = np.exp((j + 1) / (i + 1)) * np.sin((i + 1) * (j + 1))
            self.b[i] = b_i 
            
        # keep track of best guess thus far
        self.best_x = self.start
        self.max_val = max([np.dot(self.start, np.dot(self.A[i], self.start)) - np.dot(self.b[i], self.start) for i in range(self.k)])

    def maxquad_obj(self, x):
        results = [np.dot(x, np.dot(self.A[i], x)) - np.dot(self.b[i], x) for i in range(self.k)]
        return max(results)
    
    def subgrad(self, x):
        # grab index of the largest function at point x
        i_max = [np.dot(x, np.dot(self.A[i], x)) - np.dot(self.b[i], x) for i in range(self.k)].index(self.maxquad_obj(x))
        return 2 * np.dot(self.A[i_max], x) - self.b[i_max]
        
    def constant_update(self, x, iter):
        return x - (self.C / np.sqrt(iter)) * self.subgrad(x) / np.sum(self.subgrad(x) ** 2)
    
    def polyak_update(self, x, iter):
        learning_rate = (self.maxquad_obj(x) - self.max_val + (1 / iter)) / np.dot(self.subgrad(x), self.subgrad(x))
        return x - learning_rate * self.subgrad(x) / np.sum(self.subgrad(x) ** 2)
    
    def train(self):
        f_vals = []
        x = self.start
        for epoch in range(1, self.max_iter + 1):
            # take a step
            x = self.polyak_update(x, epoch)
            f_new = self.maxquad_obj(x) 
            if f_new < self.max_val:
                self.best_x = x
                self.max_val = f_new
                f_vals.append(f_new)
            else:
                f_vals.append(self.max_val)
        return f_vals
    
    def graph_log_iter(self):
        y = self.train()
        fig = plt.figure()
        plt.plot(np.log(np.arange(1, self.max_iter + 1)), np.log(y - self.max_val + 1), color='navy')
        plt.xlabel('Log of Iterations')
        plt.ylabel('Log of Sub-Optimality Gap')
        plt.show()
        fig.savefig('polyak_update.jpg', dpi=300)

In [ ]:
# constant update here
subgrad = SubgradientDescent()
results = subgrad.graph_log_iter()

# Accelerated Gradient Descent - A Symphony

asdf

In [ ]:
# accelerated gradient descent over the maxquad functions
class AGD():
    
    def __init__(self, dim=10, k=5, C=1e-10, max_iter=1000):
        """
        :param: dim - int, size of matrices A_k 
        :param: k - int, number of matrices A_1,\dots, A_k
        :param: C - float, learning rate constant C: eta = C / sqrt{t}
        :param: max_iter - int, max number of iterations allowed
        """
        self.dim = int(dim)
        self.k = int(k)
        self.C = C
        self.max_iter = int(max_iter)
        
        # starting guess for optimum
        self.start = np.ones(dim)
        
        # initialize the set of matrices A_k
        self.A = {}
        for i in range(k):
            A_i = np.zeros(shape=(dim, dim))
            for m in range(dim):
                for n in range(i):
                    A_i[m, n] = np.exp((n + 1) / (m + 1)) * np.cos((m + 1) * (n + 1)) * np.sin((i + 1))
                    A_i[n, m] = A_i[m, n]
                A_i[m, m] = (i / 10) * np.abs(np.sin(i)) + np.sum(np.abs(A_i[m, :m])) + np.sum(np.abs(A_i[m, m+1:]))
            self.A[i] = A_i

        # initialize the vectors b_k
        self.b = {}
        for i in range(k):
            b_i = np.zeros(dim)
            for j in range(dim):
                b_i[j] = np.exp((j + 1) / (i + 1)) * np.sin((i + 1) * (j + 1))
            self.b[i] = b_i 
            
        # keep track of best guess thus far
        self.best_x = self.start
        self.max_val = max([np.dot(self.start, np.dot(self.A[i], self.start)) - np.dot(self.b[i], self.start) for i in range(self.k)])

    def maxquad_obj(self, x):
        results = [np.dot(x, np.dot(self.A[i], x)) - np.dot(self.b[i], x) for i in range(self.k)]
        return max(results)
    
    def subgrad(self, x):
        # grab index of the largest function at point x
        i_max = [np.dot(x, np.dot(self.A[i], x)) - np.dot(self.b[i], x) for i in range(self.k)].index(self.maxquad_obj(x))
        return 2 * np.dot(self.A[i_max], x) - self.b[i_max]
        
    def constant_update(self, x, iter):
        return x - (self.C / np.sqrt(iter)) * self.subgrad(x) / np.sum(self.subgrad(x) ** 2)
    
    def polyak_update(self, x, iter):
        learning_rate = (self.maxquad_obj(x) - self.max_val + (1 / iter)) / np.dot(self.subgrad(x), self.subgrad(x))
        return x - learning_rate * self.subgrad(x) / np.sum(self.subgrad(x) ** 2)
    
    def train(self):
        f_vals = []
        x = self.start
        for epoch in range(1, self.max_iter + 1):
            # take a step
            x = self.polyak_update(x, epoch)
            f_new = self.maxquad_obj(x) 
            if f_new < self.max_val:
                self.best_x = x
                self.max_val = f_new
                f_vals.append(f_new)
            else:
                f_vals.append(self.max_val)
        return f_vals
    
    def graph_log_iter(self):
        y = self.train()
        fig = plt.figure()
        plt.plot(np.log(np.arange(1, self.max_iter + 1)), np.log(y - self.max_val + 1), color='navy')
        plt.xlabel('Log of Iterations')
        plt.ylabel('Log of Sub-Optimality Gap')
        plt.show()
        fig.savefig('polyak_update.jpg', dpi=300)